# Alternating Least Squares (ALS) Model

ALS (Alternating Least Squares) is a collaborative filtering algorithm commonly used in recommendation systems. It excels at matrix factorization, aiming to predict missing entries in a user-item interaction matrix. The core idea is to decompose this matrix into two lower-dimensional matrices: one capturing user preferences and the other capturing item characteristics.

**How ALS Works**

- **Matrix Factorization:**  
  Given a user-item interaction matrix \( R \) (such as ratings or watch ratios), ALS approximates \( R \) as the product of two matrices:  
  - \( U \): Each row represents a user's latent preferences.  
  - \( V \): Each row represents an item's latent features.  
  The objective is to minimize the reconstruction error between \( R \) and \( U \times V^T \).

- **Alternating Optimization:**  
  ALS works by alternately fixing one matrix (e.g., \( U \)) and solving for the other (e.g., \( V \)), repeating this process until convergence or a set number of iterations.

- **Handling Sparsity:**  
  Since real-world interaction matrices are typically sparse (with many missing values), ALS is designed to efficiently optimize using only the observed interactions.

## Fetching dataset
(can ignore this part if everyhting already loaded)

In [ ]:
%%bash
wget --no-check-certificate 'https://drive.usercontent.google.com/download?id=1qe5hOSBxzIuxBb1G_Ih5X-O65QElollE&export=download&confirm=t&uuid=b2002093-cc6e-4bd5-be47-9603f0b33470
' -O KuaiRec.zip
unzip KuaiRec.zip -d data_final_project

## Libraries Imports

In [ ]:
import os
import numpy as np
import pandas as pd
from utils import get_data_path, matrix_cleanup 

from sklearn.metrics import ndcg_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer

import plotly.express as px

DATA_PATH = get_data_path()
TRAINED_MODEL = "trained/pyspark_als_model"

# Step 1: Load the dataframes

In [ ]:
small_matrix = pd.read_csv(f"{DATA_PATH}/small_matrix.csv")

small_matrix = matrix_cleanup(small_matrix)

big_matrix = pd.read_csv(f"{DATA_PATH}/big_matrix.csv")

big_matrix = matrix_cleanup(big_matrix)


## Caption Category

In [ ]:
caption_category = pd.read_csv(f"{DATA_PATH}/kuairec_caption_category.csv", lineterminator='\n')
caption_category.head(10)

## Item category

We have various video characteristics (such as author_id, video_type, etc.), but these require minimal preprocessing.
For content-based filtering, we will utilize video features like the list of tags, applying a straightforward one-hot encoding approach.

In [21]:
# No missing values for this data
item_categories = pd.read_csv(f"{DATA_PATH}/item_categories.csv")

## Item daily features and User features

This dataset is also valuable for content-based filtering.
Since it primarily consists of textual data, we will encode the video features using a TF-IDF vectorizer.

In [ ]:
item_daily_features = pd.read_csv(f"{DATA_PATH}/item_daily_features.csv", lineterminator='\n')
item_daily_features.fillna(-1, inplace=True)

user_features = pd.read_csv(f"{DATA_PATH}/user_features.csv", lineterminator='\n')
user_features.fillna(-1, inplace=True)

# Step 2: Feature Engineering

- Extract relevant features from interactions and metadata (such as content tags and user activity history)
- Construct the user-item interaction matrix
- Optionally, derive features based on time or item popularity

#### Item categories

In [22]:
# Use MultiLabelBinarizer to manage efficiently the feat column
mlb = MultiLabelBinarizer()

# Transform the feat column to a list (evaluate with python)
item_categories["feat"] = item_categories["feat"].apply(eval)

item_categories = pd.DataFrame(mlb.fit_transform(item_categories["feat"]), 
                  columns=mlb.classes_,
                  index=item_categories["video_id"])


item_categories.reset_index(drop=True, inplace=True)
item_categories[item_categories.columns] = item_categories[item_categories.columns].astype("int16")

item_categories.head(10)


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Item daily features

We take the oldest data point for a given video_id.

Depending on the complexity, you can choose the number of features.

In [23]:
TEXT_FEATURES = ["video_type", "upload_type"] # "visible_status"
INT_FEATURES = ['video_duration','video_width', 'video_height', 'music_id', 'video_tag_id','show_cnt', 'show_user_num', 'play_cnt', 'play_user_num',
                'play_duration', 'complete_play_cnt', 'complete_play_user_num', 'valid_play_cnt', 'valid_play_user_num', 'long_time_play_cnt',
                'long_time_play_user_num', 'short_time_play_cnt', 'short_time_play_user_num', 'play_progress']

In [24]:
# Keep the latest date for each video_id
item_daily_features = item_daily_features.loc[item_daily_features.groupby("video_id")["date"].idxmax()].reset_index(drop=True)

# One-hot str features
text_daily_features = item_daily_features[TEXT_FEATURES]
onehotter = OneHotEncoder(handle_unknown="ignore")
onehot_array = onehotter.fit_transform(text_daily_features).toarray()

# Convert to DataFrame
text_daily_features = pd.DataFrame(
    onehot_array,
    columns=onehotter.get_feature_names_out(TEXT_FEATURES),
    index=item_daily_features.index)

# Merge the one-hot encoded features back into the original DataFrame
item_daily_features = pd.concat([item_daily_features[INT_FEATURES], text_daily_features], axis=1)

In [25]:
# No IDs, because it is the index
item_features_map = pd.concat([item_daily_features, item_categories], axis=1)

# Column names should be str
item_features_map.columns = item_features_map.columns.map(str)

# We can keep all the columns
item_features_columns = item_features_map.columns.tolist()

### User features

In [26]:
user_features_columns = [
    "is_lowactive_period","is_live_streamer", "is_video_author",
    "onehot_feat0", "onehot_feat1", "onehot_feat2", "onehot_feat3",
    "onehot_feat4", "onehot_feat5", "onehot_feat6", "onehot_feat7",
    "onehot_feat8", "onehot_feat9", "onehot_feat10", "onehot_feat11", 
    "onehot_feat12", "onehot_feat13", "onehot_feat14", "onehot_feat15",
    "onehot_feat16", "onehot_feat17"
]
user_features_map = user_features[user_features_columns].copy()

user_features_map[user_features_map.columns] = user_features_map[user_features_map.columns].astype("int16")

In [27]:
# Index is the associated IDs for quick creation
display(user_features_map.head(3))
display(item_features_map.head(3))

,is_lowactive_period,is_live_streamer,is_video_author,onehot_feat0,onehot_feat1,onehot_feat2,onehot_feat3,onehot_feat4,onehot_feat5,onehot_feat6,...,onehot_feat8,onehot_feat9,onehot_feat10,onehot_feat11,onehot_feat12,onehot_feat13,onehot_feat14,onehot_feat15,onehot_feat16,onehot_feat17
0,0,0,0,0,1,17,638,2,0,1,...,184,6,3,0,0,0,0,0,0,0
1,0,0,0,0,3,25,1021,0,0,1,...,186,6,2,0,0,0,0,0,0,0
2,0,0,0,0,6,8,402,0,0,0,...,51,2,3,0,0,0,0,0,0,0


,video_duration,video_width,video_height,music_id,video_tag_id,show_cnt,show_user_num,play_cnt,play_user_num,play_duration,...,21,22,23,24,25,26,27,28,29,30
0,5966.0,720,1280,3350323409,8,3710,2649,2213,1635,19547072,...,0,0,0,0,0,0,0,0,0,0
1,-1.0,886,1015,1812462382,27,30,25,8,7,94830,...,0,0,0,0,0,0,1,0,0,0
2,8000.0,720,1280,0,9,72,59,17,16,212893,...,0,0,0,0,0,0,0,0,0,0


## Dataset preparation

Environment variables

In [28]:
INTERACTION_N = 20_000_000
# Either small_matrix or big_matrix
DATASET = big_matrix
# Number of relevant items to get
K = 10
# Tolerance for considering a hit
WATCH_RATIO_THRESHOLD = 0.7

## Misc

In [29]:
print(f"Proportion of small_matrix relative to big_matrix: {small_matrix.shape[0] * 100 / big_matrix.shape[0]:.2f}%")

Proportion of small_matrix relative to big_matrix: 38.89%


# Step 3: Model architecture

We will use the ALS algorithm from pyspark.ml.recommendation with hyperparameters tuning and cross-validation.

The model is cut into 4 parts:
- Data preparation and tuning
- Model training
- Model evaluation
- Model saving


### Pyspark imports and setup

In [ ]:
import pyspark
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.sql import SparkSession
from pyspark.sql.functions import collect_list, col, expr
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel, Model

print(f"Spark version: {pyspark.__version__}")
print(f"Pandas version: {pd.__version__}")

spark = SparkSession.builder \
    .appName("KuaiRec ALS") \
    .getOrCreate()

### Data preparation for Pyspark

In [32]:
# We load directly from the CSV to avoid memory issues
small_matrix = spark.read.csv(
    f"{DATA_PATH}/small_matrix.csv",
    header=True,
    sep=",",
    nullValue="",
    # We have to infer for correct types
    inferSchema=True,
).select("user_id", "video_id", "watch_ratio").na.drop(subset=["user_id", "video_id", "watch_ratio"])

In [ ]:
# Makes a lot of problems, we will not use it for now
big_matrix = spark.read.csv(
    f"{DATA_PATH}/big_matrix.csv",
    header=True,
    sep=",",
    inferSchema=True,
    nullValue="",
).select("user_id", "video_id", "watch_ratio").na.drop(subset=["user_id", "video_id", "watch_ratio"])

### Hyperparameter tuning and Cross Validation

In [33]:
# ALS model configuration
als = ALS(
    maxIter=10,
    rank=10,
    userCol="user_id",
    itemCol="video_id",
    ratingCol="watch_ratio",
    implicitPrefs=True,
)

# For CrossValidator
params = ParamGridBuilder() \
    .addGrid(als.maxIter, [10, 15]) \
    .addGrid(als.regParam, [0.1]) \
    .build()


# RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="watch_ratio", predictionCol="prediction")


# CrossValidator
cvs = CrossValidator(
    estimator=als,
    estimatorParamMaps=params,
    evaluator=evaluator,
    # Between 2 and 5
    numFolds=3,
)

### Training
Now with the training, we should have:

R ≈ U x V

Where:
- R is the user-item interaction matrix
- U is the user feature matrix
- V is the item feature matrix

In [34]:
(training, test) = small_matrix.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# Fit the ALS model on the train data
models : CrossValidatorModel = cvs.fit(training)

### Metrics evaluation

In [36]:
# Take the best model from the CrossValidator
pyspark_als_model : Model = models.bestModel
predictions = pyspark_als_model.transform(test)
rmse = evaluator.evaluate(predictions.na.drop())

In [ ]:
print(f"RMSE: {rmse}")
print(f"Rank: {pyspark_als_model.rank}")
print(f"MaxIter: {pyspark_als_model._java_obj.parent().getMaxIter()}")
print(f"RegParam: {pyspark_als_model._java_obj.parent().getRegParam()}")

### Saving

In [ ]:
pyspark_als_model.save(TRAINED_MODEL)

# Step 4: Recommendation

- Predict which videos are likely to be enjoyed by each user in the test set
- Generate a top-N ranked list of recommendations for each user

### Loading model

In [39]:
try:
    pyspark_als_model
except NameError:
    print("Model not found. Trying to load it.")
    if os.path.exists(TRAINED_MODEL):
        print("Model found. Loading it.")
        pyspark_als_model = ALSModel.load(TRAINED_MODEL)
        print("Model loaded.")
    else:
        print("Model not found. Please train the model first.")
        raise FileNotFoundError("Model not found. Please train the model first.")


### Recommendation

In [40]:
# If you have not already used Caption Category
caption_category = pd.read_csv(f"{DATA_PATH}/kuairec_caption_category.csv", lineterminator='\n')

def video_id_to_caption(video_id: int) -> str:
    """
    Get the caption of a video from its id

    Args:
        video_id (int): The id of the video

    Returns:
        str: The caption of the video
    """

    # Get the caption from the video_id
    match = caption_category[caption_category["video_id"] == video_id]["caption"]
    # Check not NaN
    if not match.empty and match.values[0] == match.values[0]:
        return str(match.values[0])
    else:
        return "Unknown"

In [ ]:
# All user recommendations
recommends = pyspark_als_model.recommendForAllUsers(10)
recommends_df = recommends.toPandas()

# Explode to have each line as a recommendation
recommends_df = recommends_df.explode("recommendations")
recommends_df["recommendations"] = recommends_df["recommendations"].apply(
    lambda x: f"{video_id_to_caption(x[0])}")
recommends_df.set_index("user_id", inplace=True)
recommends_df

In [ ]:
# Top 5 users recommendations

# Get the top 5 users

print("Top 5 users:")
# Get the top 5 users
top_5_users = recommends_df.index.unique()[:5].tolist()
print(top_5_users)
top_users = top_5_users
top_users_recommends_df = recommends_df[recommends_df.index.isin(top_users)]
top_users_recommends_df

# Step 5: Model Evaluation

In [68]:
ground_truth = test.groupBy("user_id") \
    .agg(collect_list("video_id").alias("true_items"))

predicted = recommends.select(
    col("user_id"),
    expr("transform(recommendations, x -> x.video_id)").alias("pred_items")
)
ranking_df = predicted.join(ground_truth, on="user_id", how="inner")

ranking_pd = ranking_df.toPandas()

In [69]:
def recall_at_k(y_true, y_pred, k):
    return len(set(y_true) & set(y_pred[:k])) / len(set(y_true)) if y_true else 0

def map_at_k(y_true, y_pred, k):
    score = 0.0
    hit_count = 0.0
    for i, p in enumerate(y_pred[:k]):
        if p in y_true:
            hit_count += 1.0
            score += hit_count / (i + 1.0)
    return score / min(len(y_true), k) if y_true else 0.0

def ndcg_at_k(y_true, y_pred, k):
    relevance = [1 if item in y_true else 0 for item in y_pred[:k]]
    return ndcg_score([relevance], [relevance])
recalls, maps, ndcgs = [], [], []

In [70]:
for row in ranking_pd.itertuples():
    true_items = row.true_items
    pred_items = row.pred_items
    
    recalls.append(recall_at_k(true_items, pred_items, 10))
    maps.append(map_at_k(true_items, pred_items, 10))
    ndcgs.append(ndcg_at_k(true_items, pred_items, 10))

print(f"Recall@10: {np.mean(recalls):.4f}")
print(f"MAP@10:    {np.mean(maps):.4f}")
print(f"NDCG@10:   {np.mean(ndcgs):.4f}")


Recall@10: 0.0029
MAP@10:    0.0805
NDCG@10:   0.8760
